# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

## Store Part I results into DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame
csv_file = os.path.join('..', 'Output', 'city_weather.csv')
weather_df = pd.read_csv(csv_file) 
weather_df

,No,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,0,986717,Kruisfontein,ZA,-34.0033,24.7314,48.88,89,3,4.41
1,1,746983,Gelibolu,TR,40.4103,26.6708,76.86,76,0,1.77
2,2,2206939,Bluff,NZ,-46.6000,168.3333,49.80,96,100,9.26
3,3,537094,Lal'sk,RU,60.7415,47.5862,70.74,69,41,5.01
4,4,5122534,Jamestown,US,42.0970,-79.2353,78.28,39,1,11.01
...,...,...,...,...,...,...,...,...,...,...
554,554,3451399,Ribas do Rio Pardo,BR,-20.4431,-53.7592,80.53,32,97,8.30
555,555,186180,Moyale,KE,3.5167,39.0584,69.73,60,100,10.42
556,556,934479,Grand Gaube,MU,-20.0064,57.6608,69.39,64,93,4.00
557,557,3118304,Llanes,ES,43.4198,-4.7549,62.02,80,39,4.83


## Humidity Heatmap

In [5]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

In [9]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Temperature"] < 80) & (filtered_weather_df["Temperature"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Clouds"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()
filtered_weather_df

,No,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
1,1,746983,Gelibolu,TR,40.4103,26.6708,76.86,76,0,1.77
31,31,3460107,Jaraguá,BR,-15.7569,-49.3344,75.42,31,0,1.86
72,72,5106841,Albany,US,42.6001,-73.9662,76.39,39,0,9.55
119,119,921906,Domoni,KM,-12.2569,44.5319,73.02,78,0,3.58
188,188,921786,Mitsamiouli,KM,-11.3847,43.2844,75.42,64,0,4.23
197,197,2517117,Granada,ES,37.1882,-3.6067,72.09,54,0,1.01
245,245,1529376,Korla,CN,41.7597,86.1469,75.78,24,0,1.41
289,289,139223,Chālūs,IR,36.6550,51.4204,72.86,83,0,0.00
327,327,1142226,Dawlatabad,AF,36.4141,64.9053,77.99,17,0,6.91
339,339,300058,Susurluk,TR,39.9136,28.1578,72.28,71,0,0.83


## Hotel Map

In [ ]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [11]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 1: Gelibolu.
Closest hotel is Oya Hotel.
------------
Retrieving Results for Index 31: Jaraguá.
Closest hotel is Danbury Hotel.
------------
Retrieving Results for Index 72: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 119: Domoni.
Closest hotel is Al Djadid.
------------
Retrieving Results for Index 188: Mitsamiouli.
Closest hotel is Foyer ADM.
------------
Retrieving Results for Index 197: Granada.
Closest hotel is HOTEL REINA CRISTINA.
------------
Retrieving Results for Index 245: Korla.
Closest hotel is Gulisitan Hotel.
------------
Retrieving Results for Index 289: Chālūs.
Closest hotel is نظام مهندسي ساختمان چالوس.
------------
Retrieving Results for Index 327: Dawlatabad.
Closest hotel is WASIM ASGAR KHAN.
------------
Retrieving Results for Index 339: Susurluk.
Closest hotel is Çağan Pansiyon.
------------
Retrieving Results for Index 359: Aleksandrov Gay.
Closest hotel is Gostinitsa I.p. Yes'kov A.v.
--------

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))